<a href="https://colab.research.google.com/github/Ginika2695/data-analytics/blob/main/SOKOTO_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
election_data = pd.read_csv('/content/SOKOTO_crosschecked.csv')
print(election_data)

      State      LGA                 Ward       PU-Code  \
0    SOKOTO    BINJI               INNAME  33-01-01-002   
1    SOKOTO    BINJI               SAMAMA  33-01-02-001   
2    SOKOTO    BINJI                BINJI  33-01-03-002   
3    SOKOTO  BODINGA             BAGARAWA  33-02-02-003   
4    SOKOTO  BODINGA      DINGYADI/BADAWA  33-02-06-015   
..      ...      ...                  ...           ...   
127  SOKOTO  WAMAKKO              ARKILLA  33-21-01-039   
128  SOKOTO  WAMAKKO              ARKILLA  33-21-01-042   
129  SOKOTO  WAMAKKO  KALAMBAINA/GIRABSHI  33-21-04-012   
130  SOKOTO  WAMAKKO  KALAMBAINA/GIRABSHI  33-21-04-013   
131  SOKOTO    WURNO               MARAFA  33-22-01-017   

                                    PU-Name  Accredited_Voters  \
0                            GURUZA /SAMAMA                261   
1                             SAMAMA MARINA                244   
2                                  BAJAGAWA                303   
3                          

In [ ]:
# Z-score based outlier detection for distances (modified for numerical scores)
def calculate_distance_zscores(distances):
    return np.abs((distances - np.mean(distances)) / np.std(distances)) > 3

# IQR based outlier detection for distances (modified for numerical scores)
def calculate_distance_iqr_scores(distances):
    Q1 = np.percentile(distances, 25)
    Q3 = np.percentile(distances, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    scores = np.where((distances < lower_bound) | (distances > upper_bound),
                      np.maximum(np.abs(distances - lower_bound), np.abs(distances - upper_bound)) / IQR,
                      0)
    return scores

# Calculate Z-score based outlier scores for distances
zscore_distance_scores = calculate_distance_zscores(mean_distances)

# Calculate IQR based outlier scores for distances
iqr_distance_scores = calculate_distance_iqr_scores(mean_distances)

# Add outlier scores as new columns (overwrite previous boolean columns)
election_data['Distance_Zscore_Outlier'] = np.nan
election_data['Distance_IQR_Outlier'] = np.nan

election_data.loc[election_data['Latitude'].notna(), 'Distance_Zscore_Outlier'] = zscore_distance_scores
election_data.loc[election_data['Latitude'].notna(), 'Distance_IQR_Outlier'] = iqr_distance_scores

print(election_data)

NameError: name 'mean_distances' is not defined

In [ ]:
# Z-score based outlier detection for distances (modified for numerical scores)
def calculate_distance_zscores(distances):
    return np.abs((distances - np.mean(distances)) / np.std(distances))

# ... (rest of your existing code) ...

# Check if 'Distance' column exists before calculation
if 'Distance' in election_data.columns:
    # Calculate Z-score based outlier scores for distances (overwriting previous boolean result)
    # Ensure the result aligns with the DataFrame's length
    zscore_distance_scores = calculate_distance_zscores(election_data['Distance']) # Calculate for the entire 'Distance' column

    # Handle potential missing values in 'Distance'
    election_data['Distance_Zscore_Outlier'] = np.nan
    election_data.loc[election_data['Distance'].notna(), 'Distance_Zscore_Outlier'] = zscore_distance_scores
else:
    print("Warning: 'Distance' column not found in the DataFrame. Please calculate distances first.")

# ... (rest of your code to write to Google Sheets) ...

In [ ]:
from scipy.spatial.distance import cdist

# Calculate pairwise distances between coordinates
distances = cdist(election_data[['Latitude', 'Longitude']].dropna().values,
                 election_data[['Latitude', 'Longitude']].dropna().values)

# Calculate mean distance for each point
mean_distances = np.mean(distances, axis=1)

# Now you can use mean_distances for further calculations or analysis
# For example, you can add it as a new column to your DataFrame:
election_data['Distance'] = np.nan
election_data.loc[election_data['Latitude'].notna(), 'Distance'] = mean_distances

print(election_data)

In [ ]:
# Outlier Detection using Z-score
def calculate_zscores(data):
    return np.abs((data - data.mean()) / data.std())

# Outlier Detection using IQR
def calculate_iqr_scores(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    scores = np.where((data < lower_bound) | (data > upper_bound),
                      np.maximum(np.abs(data - lower_bound), np.abs(data - upper_bound)) / IQR,
                      0)
    return scores

# Calculate outlier scores and add them as columns
for party in ['LP', 'APC', 'PDP', 'NNPP']:
    election_data[f'{party}_Zscore_Outlier'] = calculate_zscores(election_data[party])
    election_data[f'{party}_IQR_Outlier'] = calculate_iqr_scores(election_data[party])

print(election_data)

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Ensure the spreadsheet name is correct and exists in your Google Drive
spreadsheet_name = 'Sokoto updated sheet'  # Double-check this name

try:
    # Attempt to open the spreadsheet
    sheet = gc.open(spreadsheet_name).sheet1

    # Convert the DataFrame to a list of lists for writing to Google Sheets
    # Replace infinite values with a large number or string representation
    data_to_write = election_data.replace([np.inf, -np.inf], np.nan).fillna('Infinity').values.tolist()

    # Write the data to the Google Sheet (starting from cell A1)
    sheet.update('A1', data_to_write)

    print("Data successfully written to Google Sheet!")

except gspread.exceptions.SpreadsheetNotFound:
    print(f"Error: Spreadsheet '{spreadsheet_name}' not found. Please check the name and ensure it exists.")

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Open the desired Google Sheet (or create a new one)
# Replace 'Your Spreadsheet Name' with the actual name of your sheet
sheet = gc.open('Sokoto updated sheet').sheet1

# Get the column names
column_names = election_data.columns.tolist()

# Convert the DataFrame to a list of lists, including column names
# Replace infinite and NaN values with empty strings
# (or any other suitable representation for your use case)
data_to_write = [column_names] + election_data.replace([np.inf, -np.inf, np.nan], '').values.tolist()

# Write the data to the Google Sheet (starting from cell A1)
sheet.update('A1', data_to_write)

In [ ]:
# Outlier Detection using Z-score
def calculate_zscores(data):
    return np.abs((data - data.mean()) / data.std())

# Outlier Detection using IQR
def calculate_iqr_scores(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    scores = np.where((data < lower_bound) | (data > upper_bound),
                      np.maximum(np.abs(data - lower_bound), np.abs(data - upper_bound)) / IQR,
                      0)
    return scores

# Calculate outlier scores and add them as columns
# Make sure to execute this cell before running the outlier detection code
for party in ['LP', 'APC', 'PDP', 'NNPP']:
    election_data[f'{party}_Zscore_Outlier'] = calculate_zscores(election_data[party])
    election_data[f'{party}_IQR_Outlier'] = calculate_iqr_scores(election_data[party])

# Now you can proceed with outlier detection
for party in ['LP', 'PDP', 'APC', 'NNPP']:
    # Filter for outliers based on Z-score and sort by outlier score
    party_outliers = election_data[election_data[f'{party}_Zscore_Outlier'] > 3].sort_values(by=f'{party}_Zscore_Outlier', ascending=False).head(3)
    print(f"Top 3 Outliers for {party} (Z-score):\n", party_outliers[['Ward', party, f'{party}_Zscore_Outlier']])

In [ ]:
# ... previous code for outlier calculation ...

# Authenticate and authorize with Google Sheets
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)  # Define 'gc' here

# Open the desired Google Sheet and select sheet 2
sheet2 = gc.open('Sokoto updated sheet').get_worksheet(1)  # 0-indexed, so 1 is the second sheet

# ... (rest of your code) ...